# EDA

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:.4f}'.format)

In [3]:
df = pd.read_csv('AA_mintafeladat_converted.csv')

In [4]:
df.head()

,Key,start_of_study,modul_code,study_status,day_or_night,level_of_degree,financial_status,semester_status,sex,date_of_birth,post_code,semester_num,active_semesters,taken_credit,cum_taken_credit,comp_credits,cum_comp_credits,avg,cumulated_avg,academic_year,semester_number
0,1,2010.09.10,ETNEMBSC,Aktív,Nappali,alapképzés (BA/BSc),Államilag finanszírozott,Aktív,Nő,1991-12-01,1126,1,7.0000,33.0000,33.0000,33.0000,33.0000,3.6400,3.6400,2010-2011,1
1,1,2010.09.10,ETNEMBSC,Aktív,Nappali,alapképzés (BA/BSc),Államilag finanszírozott,Aktív,Nő,1991-12-01,1126,2,7.0000,31.0000,64.0000,31.0000,64.0000,4.7100,4.1600,2010-2011,2
2,1,2010.09.10,ETNEMBSC,Aktív,Nappali,alapképzés (BA/BSc),Államilag finanszírozott,Aktív,Nő,1991-12-01,1126,3,7.0000,30.0000,94.0000,30.0000,94.0000,4.2700,4.1900,2011-2012,1
3,1,2010.09.10,ETNEMBSC,Aktív,Nappali,alapképzés (BA/BSc),Államilag finanszírozott,Aktív,Nő,1991-12-01,1126,4,7.0000,31.0000,125.0000,31.0000,125.0000,4.0600,4.1600,2011-2012,2
4,1,2010.09.10,ETNEMBSC,Aktív,Nappali,alapképzés (BA/BSc),Államilag finanszírozott,Aktív,Nő,1991-12-01,1126,5,7.0000,31.0000,156.0000,31.0000,156.0000,4.1600,4.1600,2012-2013,1


In [5]:
print("=== UNIVERSITY STUDENT DATA VALIDATION ===")
print(f"Dataset: {df.shape[0]:,} rows, {df.shape[1]} columns")
print(f"Date range: {df['academic_year'].min()} to {df['academic_year'].max()}")


=== UNIVERSITY STUDENT DATA VALIDATION ===
Dataset: 63,262 rows, 21 columns
Date range: 2005-2006 to 2013-2014


In [6]:
degree_cols = ['Key', 'modul_code', 'study_status', 'active_semesters']
print("number of degrees:", df[degree_cols].drop_duplicates().shape[0])
print("unique students:", df['Key'].nunique())
print("modul codes:", df['modul_code'].nunique())

number of degrees: 14583
unique students: 12971
modul codes: 26


In [7]:
degree_groups = df.groupby(degree_cols)

validation_issues = {
    'monotonic_cum_taken_credit': [],
    'monotonic_cum_comp_credits': [],
    'monotonic_cumulated_avg': [],
    'sum_mismatch_taken_credit': [],
    'sum_mismatch_comp_credits': [],
    'inconsistent_active_semesters': []
}

print("cumulative value validation for each degree")

for degree_key, group in degree_groups:
    # sort by semester_num to - chronological order
    group_sorted = group.sort_values('semester_num')
    
    # monotonic increase for cumulative values
    cum_taken_credit = group_sorted['cum_taken_credit'].values
    cum_comp_credits = group_sorted['cum_comp_credits'].values
    cumulated_avg = group_sorted['cumulated_avg'].values
    
    # allowing for equal consecutive values
    if not all(cum_taken_credit[i] >= cum_taken_credit[i-1] for i in range(1, len(cum_taken_credit))):
        validation_issues['monotonic_cum_taken_credit'].append(degree_key)
    
    if not all(cum_comp_credits[i] >= cum_comp_credits[i-1] for i in range(1, len(cum_comp_credits))):
        validation_issues['monotonic_cum_comp_credits'].append(degree_key)
    
    #  cumulative values should match sum of non-cumulative
    expected_cum_taken = group_sorted['taken_credit'].cumsum().values
    expected_cum_comp = group_sorted['comp_credits'].cumsum().values
    
    if not np.allclose(cum_taken_credit, expected_cum_taken, equal_nan=True):
        validation_issues['sum_mismatch_taken_credit'].append((degree_key, cum_taken_credit.tolist(), expected_cum_taken.tolist()))
    
    if not np.allclose(cum_comp_credits, expected_cum_comp, equal_nan=True):
        validation_issues['sum_mismatch_comp_credits'].append((degree_key, cum_comp_credits.tolist(), expected_cum_comp.tolist()))
    
    # active_semesters consistency within degree
    if group['active_semesters'].nunique() > 1:
        validation_issues['inconsistent_active_semesters'].append((degree_key, group['active_semesters'].unique()))
print("VALIDATION RESULTS:")
for issue_type, issues in validation_issues.items():
    print(f"{issue_type}: {len(issues)} degrees with issues")


cumulative value validation for each degree
VALIDATION RESULTS:
monotonic_cum_taken_credit: 0 degrees with issues
monotonic_cum_comp_credits: 0 degrees with issues
monotonic_cumulated_avg: 0 degrees with issues
sum_mismatch_taken_credit: 0 degrees with issues
sum_mismatch_comp_credits: 0 degrees with issues
inconsistent_active_semesters: 0 degrees with issues


In [8]:
missing_data = df.isnull().sum()
print(f"Missing values per column:")
for col, missing in missing_data.items():
    if missing > 0:
        print(f"  {col}: {missing} ({missing/len(df)*100:.1f}%)")


Missing values per column:


In [9]:
status_dist = df['study_status'].value_counts()
print("Study status distribution:")
for status, count in status_dist.items():
    print(f"  {status}: {count:,} ({count/len(df)*100:.1f}%)")

Study status distribution:
  Aktív: 26,768 (42.3%)
  Diplomát szerzett: 19,754 (31.2%)
  Törölt: 9,619 (15.2%)
  Passzív: 3,552 (5.6%)
  Abszolvált: 1,618 (2.6%)
  Záróvizsgát tett: 981 (1.6%)
  Szakváltoztató: 515 (0.8%)
  Sikeres záróvizsga: 227 (0.4%)
  Képzés váltó: 49 (0.1%)
  Részismeretes(végzett): 40 (0.1%)
  Még nem meghatározott: 26 (0.0%)
  Elhunyt: 24 (0.0%)
  Tagozat váltó: 23 (0.0%)
  Bejelentés nélkül távozott: 17 (0.0%)
  Aktív vendéghallgató: 14 (0.0%)
  Saját kérésre törölve: 13 (0.0%)
  Intézményváltoztató: 10 (0.0%)
  Sikertelen záróvizsga: 6 (0.0%)
  Felvett: 3 (0.0%)
  Nem beszámító aktív: 2 (0.0%)
  Átsorolva más jelű tantervbe: 1 (0.0%)


In [10]:
status_dist = df['financial_status'].value_counts()
print("financial_status distribution:")
for status, count in status_dist.items():
    print(f"  {status}: {count:,} ({count/len(df)*100:.1f}%)")

financial_status distribution:
  Államilag támogatott: 20,576 (32.5%)
  Államilag finanszírozott: 16,516 (26.1%)
  Költségtérítéses: 16,204 (25.6%)
  Állami ösztöndíjas: 4,688 (7.4%)
  Önköltséges: 3,997 (6.3%)
  Miniszteri ösztöndíjas: 291 (0.5%)
  Állami részösztöndíjas: 270 (0.4%)
  Államilag finanszírozott (képzési időn túli): 225 (0.4%)
  Költségtérítéses (képzési időn túli): 137 (0.2%)
  Költségtérítéses (másoddiplomás): 98 (0.2%)
  A1 kategóriás külföldi ösztöndíjas: 65 (0.1%)
  B külf. külső tám. ösztöndíj: 46 (0.1%)
  Részképzés: 21 (0.0%)
  C kategóriás külföldi ösztöndíjas: 20 (0.0%)
  Államilag támogatott (képzési időn túli): 17 (0.0%)
  Költségtérítéses (vendéghallgató): 17 (0.0%)
  Még nem meghatározott: 17 (0.0%)
  Költségtérítés mentes (GYES...): 13 (0.0%)
  Államilag finanszírozott túllépő: 13 (0.0%)
  Államilag támogatott (vendéghallgató): 12 (0.0%)
  Költségtérítés mentes: 8 (0.0%)
  Normatív támogatással A3 HTM: 4 (0.0%)
  Költségtérítéses (párhuzamos): 3 (0.0%)
  E

In [11]:
status_dist = df['semester_status'].value_counts()
print("semester_status distribution:")
for status, count in status_dist.items():
    print(f"  {status}: {count:,} ({count/len(df)*100:.1f}%)")

semester_status distribution:
  Aktív: 55,964 (88.5%)
  Passzív: 6,732 (10.6%)
  Még nem meghatározott: 181 (0.3%)
  Törölt: 139 (0.2%)
  Aktív vendéghallgató: 84 (0.1%)
  Felvett: 40 (0.1%)
  Akkreditált: 33 (0.1%)
  Abszolvált: 27 (0.0%)
  Áthallgató: 21 (0.0%)
  Részismeretes(végzett): 12 (0.0%)
  Szakváltoztató: 9 (0.0%)
  Nem beszámító aktív: 4 (0.0%)
  Évhalasztó: 3 (0.0%)
  Képzés váltó: 3 (0.0%)
  Passzív vendéghallgató: 2 (0.0%)
  Átsorolva más jelű tantervbe: 2 (0.0%)
  Szünetel: 2 (0.0%)
  Záróvizsgára vár: 1 (0.0%)
  Elhunyt: 1 (0.0%)
  Bejelentés nélkül távozott: 1 (0.0%)
  Saját kérésre törölve: 1 (0.0%)


In [12]:
status_dist = df['day_or_night'].value_counts()
print("day_or_night distribution:")
for status, count in status_dist.items():
    print(f"  {status}: {count:,} ({count/len(df)*100:.1f}%)")

day_or_night distribution:
  Nappali: 54,230 (85.7%)
  Levelező: 9,032 (14.3%)


In [13]:
# fig, axes = plt.subplots(3, 2, figsize=(20, 10))

# # Target variable distribution
# df['study_status'].value_counts().plot(kind='bar', ax=axes[0,0])
# axes[0,0].set_title('Distribution of study status')
# axes[0,0].tick_params(axis='x')

# # Grade distribution
# df['avg'].hist(bins=30, ax=axes[0,1])
# axes[0,1].set_title('Distribution of average grades')

# # Credits distribution
# df['cum_taken_credit'].hist(bins=30, ax=axes[1,0])
# axes[1,0].set_title('Distribution of cumulative credits')

# # Semester number distribution
# df['semester_num'].value_counts().sort_index().plot(kind='bar', ax=axes[1, 1])
# axes[1, 1].set_title('Distribution by semester number')

# df['taken_credit'].value_counts().sort_index().plot(kind='bar', ax=axes[2, 0])
# axes[2, 0].set_title('Distribution of taken credit')
# axes[2, 0].set_xlim(-0.5, 50)
# # axes[2, 0].set_xticks([0, 25, 50]) 

# df['taken_credit'].value_counts().sort_index().plot(kind='bar', ax=axes[2, 1])
# axes[2, 1].set_title('Distribution of taken credit')
# axes[2, 1].set_xticks([0, 50, 100, 150, 200, 250]) 


# # fig.delaxes(axes[2, 1]) 

# plt.tight_layout()
plt.show()

# FEATURE ENGINEEERING

In [14]:
group_cols = ['Key', 'modul_code', 'study_status', 'active_semesters']

In [ ]:
status_mapping = {
    'Aktív': 'active',
    'Aktív vendéghallgató': 'active',
    'Felvett': 'active',
    'Nem beszámító aktív': 'active',


    'Diplomát szerzett': 'graduated', 
    'Sikeres záróvizsga': 'graduated',
    'Részismeretes(végzett)': 'graduated',
    'Záróvizsgát tett': 'graduated',
    'Abszolvált': 'graduated',

    'Passzív': 'at_risk',
    'Sikertelen záróvizsga': 'at_risk',
    'Még nem meghatározott': 'at_risk',

    'Saját kérésre törölve': 'dropped_out',
    'Törölt': 'dropped_out',
    'Bejelentés nélkül távozott': 'dropped_out',
    'Elhunyt': 'dropped_out',
    
    'Tagozat váltó': 'transferred',
    'Szakváltoztató': 'transferred',
    'Képzés váltó': 'transferred',
    'Intézményváltoztató': 'transferred',
    'Átsorolva más jelű tantervbe': 'transferred'
}

df['status_category'] = df['study_status'].map(status_mapping)


In [16]:
df['churn_target'] = (df['status_category'] == 'dropped_out').astype(int)
print(f"churn rate: {df['churn_target'].mean():.1%}")

churn rate: 15.3%


In [17]:
passive_dict = ["Passzív", "Még nem meghatározott", "Törölt", "Passzív vendéghallgató","Szünetel",
                "Elhunyt", "Bejelentés nélkül távozott", "Saját kérésre törölve"]
df['is_passive'] = (df['semester_status'].isin(passive_dict)).astype(int)

In [18]:
df['start_of_study'] = pd.to_datetime(df['start_of_study'], format='%Y.%m.%d')
df['date_of_birth'] = pd.to_datetime(df['date_of_birth'])


In [19]:
state_funded =  ["Államilag támogatott", "Államilag finanszírozott", "Állami ösztöndíjas", "Miniszteri ösztöndíjas", 
"Állami részösztöndíjas", "Államilag finanszírozott (képzési időn túli)", "A1 kategóriás külföldi ösztöndíjas", "B külf. külső tám. ösztöndíj",
"C kategóriás külföldi ösztöndíjas", "Államilag támogatott (képzési időn túli)", "Költségtérítés mentes (GYES...)", "Államilag finanszírozott túllépő",
"Államilag támogatott (vendéghallgató)", "Költségtérítés mentes", "Normatív támogatással A3 HTM", "A3R Részképzős HTM normatív támogatással"]

df['is_state_funded'] = (df['financial_status'].isin(state_funded)).astype(int)
# df.head(50)

In [20]:
agg_dict = {
    'Key': 'first',
    # Target
    'churn_target': 'first',
    
    # Existing features
    'day_or_night': 'first',
    'level_of_degree': 'first', 
    'sex': 'first',
    'active_semesters': 'first',
    
    # Dates for engineering
    'start_of_study': 'first',
    'date_of_birth': 'first',
    'post_code': 'first',
    
    # Financial
    'is_state_funded': ['first', 'last', 'mean'],
    
    # Academic
    'semester_num': ['count'],
    'academic_year': 'nunique',
    
    # Passive tracking
    'is_passive': 'sum',
    
    # Performance
    'avg': ['first', 'last', 'mean'],
    'taken_credit': 'sum',
    'cum_taken_credit': 'last',  
    'comp_credits': 'sum',       
    'cum_comp_credits': 'last',  
    'cumulated_avg': 'last'      
}

features = df.groupby(group_cols).agg(agg_dict)
features.columns = ['_'.join(col) if isinstance(col, tuple) else col for col in features.columns]
features = features.reset_index()

In [21]:
features

,Key,modul_code,study_status,active_semesters,Key_first,churn_target_first,day_or_night_first,level_of_degree_first,sex_first,active_semesters_first,start_of_study_first,date_of_birth_first,post_code_first,is_state_funded_first,is_state_funded_last,is_state_funded_mean,semester_num_count,academic_year_nunique,is_passive_sum,avg_first,avg_last,avg_mean,taken_credit_sum,cum_taken_credit_last,comp_credits_sum,cum_comp_credits_last,cumulated_avg_last
0,1,ETNEMBSC,Aktív,7.0000,1,0,Nappali,alapképzés (BA/BSc),Nő,7.0000,2010-09-10,1991-12-01,1126,1,1,1.0000,7,4,0,3.6400,4.7000,4.3200,206.0000,206.0000,206.0000,206.0000,4.2900
1,2,ETNEMBSC,Diplomát szerzett,7.0000,2,0,Nappali,alapképzés (BA/BSc),Nő,7.0000,2008-09-12,1990-02-01,4031,0,1,0.8571,7,4,0,2.8800,4.0000,3.7200,201.0000,201.0000,195.0000,195.0000,3.6700
2,3,3MLKERT,Törölt,6.0000,3,1,Levelező,mesterképzés (MA/MSc),Férfi,6.0000,2008-09-09,1983-04-01,3036,0,0,0.4000,10,5,4,3.6000,0.0000,1.9490,184.0000,184.0000,99.0000,99.0000,3.7000
3,4,7BNKOMM,Aktív,1.0000,4,0,Nappali,alapképzés (BA/BSc),Férfi,1.0000,2013-09-05,1994-08-01,9400,0,0,0.0000,1,1,0,0.0000,0.0000,0.0000,31.0000,31.0000,24.0000,24.0000,0.0000
4,5,2BNGINFB,Aktív,1.0000,5,0,Nappali,alapképzés (BA/BSc),Férfi,1.0000,2013-09-04,1992-06-01,1158,1,1,1.0000,1,1,0,0.0000,0.0000,0.0000,79.0000,79.0000,63.0000,63.0000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14578,12968,3BNKEMN,Törölt,8.0000,12968,1,Nappali,alapképzés (BA/BSc),Nő,8.0000,2008-07-29,1990-01-01,3252,1,1,1.0000,8,4,0,3.0700,0.0000,2.7662,248.0000,248.0000,164.0000,164.0000,3.3500
14579,12969,3BLKEMN,Aktív,1.0000,12969,0,Levelező,alapképzés (BA/BSc),Nő,1.0000,2012-10-11,1984-12-01,1152,0,0,0.0000,3,2,2,0.0000,2.8000,0.9333,147.0000,147.0000,138.0000,138.0000,2.8000
14580,12969,3BNKEMN,Törölt,4.0000,12969,1,Nappali,alapképzés (BA/BSc),Nő,4.0000,2010-09-22,1984-12-01,1152,0,0,0.2000,5,3,1,3.8200,2.0000,3.3580,129.0000,129.0000,78.0000,78.0000,4.2800
14581,12970,7BNKOMM,Aktív,1.0000,12970,0,Nappali,alapképzés (BA/BSc),Férfi,1.0000,2013-09-05,1994-12-01,1028,0,0,0.0000,1,1,0,0.0000,0.0000,0.0000,31.0000,31.0000,24.0000,24.0000,0.0000


In [22]:
features['age_at_start'] = ((features['start_of_study_first'] - features['date_of_birth_first']).dt.days / 365.25).round(1)
# features.head()

In [23]:
features['is_fall_start'] = (features['start_of_study_first'].dt.month >= 9).astype(int)
# features.head(50)

In [24]:
features['semesters_skipped'] = features['semester_num_count'] - features['active_semesters_first'] 
features['semesters_skipped'] = features['semesters_skipped'].clip(lower=0)  # Only positive values
features['has_gap'] = (features['semesters_skipped'] > 0).astype(int)
features[features['semesters_skipped']> 0]

features['activity_ratio'] = features['active_semesters_first'] / features['semester_num_count']
features

,Key,modul_code,study_status,active_semesters,Key_first,churn_target_first,day_or_night_first,level_of_degree_first,sex_first,active_semesters_first,start_of_study_first,date_of_birth_first,post_code_first,is_state_funded_first,is_state_funded_last,is_state_funded_mean,semester_num_count,academic_year_nunique,is_passive_sum,avg_first,avg_last,avg_mean,taken_credit_sum,cum_taken_credit_last,comp_credits_sum,cum_comp_credits_last,cumulated_avg_last,age_at_start,is_fall_start,semesters_skipped,has_gap,activity_ratio
0,1,ETNEMBSC,Aktív,7.0000,1,0,Nappali,alapképzés (BA/BSc),Nő,7.0000,2010-09-10,1991-12-01,1126,1,1,1.0000,7,4,0,3.6400,4.7000,4.3200,206.0000,206.0000,206.0000,206.0000,4.2900,18.8000,1,0.0000,0,1.0000
1,2,ETNEMBSC,Diplomát szerzett,7.0000,2,0,Nappali,alapképzés (BA/BSc),Nő,7.0000,2008-09-12,1990-02-01,4031,0,1,0.8571,7,4,0,2.8800,4.0000,3.7200,201.0000,201.0000,195.0000,195.0000,3.6700,18.6000,1,0.0000,0,1.0000
2,3,3MLKERT,Törölt,6.0000,3,1,Levelező,mesterképzés (MA/MSc),Férfi,6.0000,2008-09-09,1983-04-01,3036,0,0,0.4000,10,5,4,3.6000,0.0000,1.9490,184.0000,184.0000,99.0000,99.0000,3.7000,25.4000,1,4.0000,1,0.6000
3,4,7BNKOMM,Aktív,1.0000,4,0,Nappali,alapképzés (BA/BSc),Férfi,1.0000,2013-09-05,1994-08-01,9400,0,0,0.0000,1,1,0,0.0000,0.0000,0.0000,31.0000,31.0000,24.0000,24.0000,0.0000,19.1000,1,0.0000,0,1.0000
4,5,2BNGINFB,Aktív,1.0000,5,0,Nappali,alapképzés (BA/BSc),Férfi,1.0000,2013-09-04,1992-06-01,1158,1,1,1.0000,1,1,0,0.0000,0.0000,0.0000,79.0000,79.0000,63.0000,63.0000,0.0000,21.3000,1,0.0000,0,1.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14578,12968,3BNKEMN,Törölt,8.0000,12968,1,Nappali,alapképzés (BA/BSc),Nő,8.0000,2008-07-29,1990-01-01,3252,1,1,1.0000,8,4,0,3.0700,0.0000,2.7662,248.0000,248.0000,164.0000,164.0000,3.3500,18.6000,0,0.0000,0,1.0000
14579,12969,3BLKEMN,Aktív,1.0000,12969,0,Levelező,alapképzés (BA/BSc),Nő,1.0000,2012-10-11,1984-12-01,1152,0,0,0.0000,3,2,2,0.0000,2.8000,0.9333,147.0000,147.0000,138.0000,138.0000,2.8000,27.9000,1,2.0000,1,0.3333
14580,12969,3BNKEMN,Törölt,4.0000,12969,1,Nappali,alapképzés (BA/BSc),Nő,4.0000,2010-09-22,1984-12-01,1152,0,0,0.2000,5,3,1,3.8200,2.0000,3.3580,129.0000,129.0000,78.0000,78.0000,4.2800,25.8000,1,1.0000,1,0.8000
14581,12970,7BNKOMM,Aktív,1.0000,12970,0,Nappali,alapképzés (BA/BSc),Férfi,1.0000,2013-09-05,1994-12-01,1028,0,0,0.0000,1,1,0,0.0000,0.0000,0.0000,31.0000,31.0000,24.0000,24.0000,0.0000,18.8000,1,0.0000,0,1.0000


In [25]:
features['financial_status_changed'] = (features['is_state_funded_first'] != features['is_state_funded_last']).astype(int)

In [26]:
features['is_from_budapest'] = ((features['post_code_first'].astype(str).str.len() == 4) & (features['post_code_first'].astype(str).str[0] == '1')).astype(int)

In [27]:
# avg related
features['grade_trajectory'] = features['avg_last'] - features['avg_first']
features['started_strong'] = (features['avg_first'] >= 3.5).astype(int)
features['started_weak'] = (features['avg_first'] < 2.5).astype(int)
features['finished_strong'] = (features['avg_last'] >= 3.5).astype(int)

features['weighted_avg'] = (features['avg_first'] * 0.3 + features['avg_last'] * 0.7)

# features['grade_volatility'] = features.groupby('Key')['avg_mean'].transform('std')
features['improvement_rate'] = features['grade_trajectory'] / features['active_semesters_first']

In [28]:
# credits-related
features['credits_per_semester'] = features['cum_comp_credits_last'] / features['active_semesters_first']
features['credit_efficiency'] = features['cum_comp_credits_last'] / features['cum_taken_credit_last']


In [29]:
# Cumulative performance trends
features['cumulative_stability'] = abs(features['cumulated_avg_last'] - features['avg_first'])
features['performance_gap'] = features['cumulated_avg_last'] - features['avg_last']

# Credit-GPA interaction
features['efficiency_score'] = (features['cum_comp_credits_last'] / features['cum_taken_credit_last']) * features['cumulated_avg_last']



In [30]:
features['is_remote'] = (features['day_or_night_first'] == 'Levelező').astype(int)
features['is_bachelor'] = (features['level_of_degree_first'] == 'alapképzés (BA/BSc)').astype(int)
features['is_female'] = (features['sex_first'] == 'Nő').astype(int)

In [31]:
features

,Key,modul_code,study_status,active_semesters,Key_first,churn_target_first,day_or_night_first,level_of_degree_first,sex_first,active_semesters_first,start_of_study_first,date_of_birth_first,post_code_first,is_state_funded_first,is_state_funded_last,is_state_funded_mean,semester_num_count,academic_year_nunique,is_passive_sum,avg_first,avg_last,avg_mean,taken_credit_sum,cum_taken_credit_last,comp_credits_sum,cum_comp_credits_last,cumulated_avg_last,age_at_start,is_fall_start,semesters_skipped,has_gap,activity_ratio,financial_status_changed,is_from_budapest,grade_trajectory,started_strong,started_weak,finished_strong,weighted_avg,improvement_rate,credits_per_semester,credit_efficiency,cumulative_stability,performance_gap,efficiency_score,is_remote,is_bachelor,is_female
0,1,ETNEMBSC,Aktív,7.0000,1,0,Nappali,alapképzés (BA/BSc),Nő,7.0000,2010-09-10,1991-12-01,1126,1,1,1.0000,7,4,0,3.6400,4.7000,4.3200,206.0000,206.0000,206.0000,206.0000,4.2900,18.8000,1,0.0000,0,1.0000,0,1,1.0600,1,0,1,4.3820,0.1514,29.4286,1.0000,0.6500,-0.4100,4.2900,0,1,1
1,2,ETNEMBSC,Diplomát szerzett,7.0000,2,0,Nappali,alapképzés (BA/BSc),Nő,7.0000,2008-09-12,1990-02-01,4031,0,1,0.8571,7,4,0,2.8800,4.0000,3.7200,201.0000,201.0000,195.0000,195.0000,3.6700,18.6000,1,0.0000,0,1.0000,1,0,1.1200,0,0,1,3.6640,0.1600,27.8571,0.9701,0.7900,-0.3300,3.5604,0,1,1
2,3,3MLKERT,Törölt,6.0000,3,1,Levelező,mesterképzés (MA/MSc),Férfi,6.0000,2008-09-09,1983-04-01,3036,0,0,0.4000,10,5,4,3.6000,0.0000,1.9490,184.0000,184.0000,99.0000,99.0000,3.7000,25.4000,1,4.0000,1,0.6000,0,0,-3.6000,1,0,0,1.0800,-0.6000,16.5000,0.5380,0.1000,3.7000,1.9908,1,0,0
3,4,7BNKOMM,Aktív,1.0000,4,0,Nappali,alapképzés (BA/BSc),Férfi,1.0000,2013-09-05,1994-08-01,9400,0,0,0.0000,1,1,0,0.0000,0.0000,0.0000,31.0000,31.0000,24.0000,24.0000,0.0000,19.1000,1,0.0000,0,1.0000,0,0,0.0000,0,1,0,0.0000,0.0000,24.0000,0.7742,0.0000,0.0000,0.0000,0,1,0
4,5,2BNGINFB,Aktív,1.0000,5,0,Nappali,alapképzés (BA/BSc),Férfi,1.0000,2013-09-04,1992-06-01,1158,1,1,1.0000,1,1,0,0.0000,0.0000,0.0000,79.0000,79.0000,63.0000,63.0000,0.0000,21.3000,1,0.0000,0,1.0000,0,1,0.0000,0,1,0,0.0000,0.0000,63.0000,0.7975,0.0000,0.0000,0.0000,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14578,12968,3BNKEMN,Törölt,8.0000,12968,1,Nappali,alapképzés (BA/BSc),Nő,8.0000,2008-07-29,1990-01-01,3252,1,1,1.0000,8,4,0,3.0700,0.0000,2.7662,248.0000,248.0000,164.0000,164.0000,3.3500,18.6000,0,0.0000,0,1.0000,0,0,-3.0700,0,0,0,0.9210,-0.3837,20.5000,0.6613,0.2800,3.3500,2.2153,0,1,1
14579,12969,3BLKEMN,Aktív,1.0000,12969,0,Levelező,alapképzés (BA/BSc),Nő,1.0000,2012-10-11,1984-12-01,1152,0,0,0.0000,3,2,2,0.0000,2.8000,0.9333,147.0000,147.0000,138.0000,138.0000,2.8000,27.9000,1,2.0000,1,0.3333,0,1,2.8000,0,1,0,1.9600,2.8000,138.0000,0.9388,2.8000,0.0000,2.6286,1,1,1
14580,12969,3BNKEMN,Törölt,4.0000,12969,1,Nappali,alapképzés (BA/BSc),Nő,4.0000,2010-09-22,1984-12-01,1152,0,0,0.2000,5,3,1,3.8200,2.0000,3.3580,129.0000,129.0000,78.0000,78.0000,4.2800,25.8000,1,1.0000,1,0.8000,0,1,-1.8200,1,0,0,2.5460,-0.4550,19.5000,0.6047,0.4600,2.2800,2.5879,0,1,1
14581,12970,7BNKOMM,Aktív,1.0000,12970,0,Nappali,alapképzés (BA/BSc),Férfi,1.0000,2013-09-05,1994-12-01,1028,0,0,0.0000,1,1,0,0.0000,0.0000,0.0000,31.0000,31.0000,24.0000,24.0000,0.0000,18.8000,1,0.0000,0,1.0000,0,1,0.0000,0,1,0,0.0000,0.0000,24.0000,0.7742,0.0000,0.0000,0.0000,0,1,0


In [32]:
features['credits_per_semester'] = features['cum_comp_credits_last'] / features['active_semesters_first']
features['credits_per_semester'] = features['credits_per_semester'].replace([np.inf, -np.inf, np.nan], 0)

features['credit_efficiency'] = features['cum_comp_credits_last'] / features['cum_taken_credit_last']
features['credit_efficiency'] = features['credit_efficiency'].replace([np.inf, -np.inf, np.nan], 0)

features['improvement_rate'] = features['grade_trajectory'] / features['active_semesters_first']
features['improvement_rate'] = features['improvement_rate'].replace([np.inf, -np.inf, np.nan], 0)

features['efficiency_score'] = (features['cum_comp_credits_last'] / features['cum_taken_credit_last']) * features['cumulated_avg_last']
features['efficiency_score'] = features['efficiency_score'].replace([np.inf, -np.inf, np.nan], 0)

features['activity_ratio'] = features['active_semesters_first'] / features['semester_num_count']
features['activity_ratio'] = features['activity_ratio'].replace([np.inf, -np.inf], 0)

In [33]:
print(f"Infinity values: {np.isinf(features.select_dtypes(include=[np.number])).sum().sum()}")
print(f"NaN values: {features.select_dtypes(include=[np.number]).isnull().sum().sum()}")

print("\nNaN count per numeric column:")
print(features.select_dtypes(include=[np.number]).isnull().sum())

# Show rows where any numeric column has NaN
print("\nRows with NaNs in numeric columns:")
print(features[features.select_dtypes(include=[np.number]).isnull().any(axis=1)])

Infinity values: 0
NaN values: 0

NaN count per numeric column:
Key                         0
active_semesters            0
Key_first                   0
churn_target_first          0
active_semesters_first      0
is_state_funded_first       0
is_state_funded_last        0
is_state_funded_mean        0
semester_num_count          0
academic_year_nunique       0
is_passive_sum              0
avg_first                   0
avg_last                    0
avg_mean                    0
taken_credit_sum            0
cum_taken_credit_last       0
comp_credits_sum            0
cum_comp_credits_last       0
cumulated_avg_last          0
age_at_start                0
is_fall_start               0
semesters_skipped           0
has_gap                     0
activity_ratio              0
financial_status_changed    0
is_from_budapest            0
grade_trajectory            0
started_strong              0
started_weak                0
finished_strong             0
weighted_avg                0
improv

In [34]:
features['degree_id'] = features['Key'].astype(str) + '_' + features['modul_code'] + '_' + features['study_status'] + '_' + features['active_semesters'].astype(str)
final_features = [
    'degree_id',
    'churn_target_first',

    'is_remote', 'is_bachelor', 'is_female', 'active_semesters_first',
    
    'is_state_funded_mean',  'age_at_start', 'is_fall_start', 
    'has_gap', 'is_state_funded_first',
    'financial_status_changed', 'is_from_budapest',
    'activity_ratio', 'grade_trajectory', 'started_strong', 'started_weak',
    'credits_per_semester', 'credit_efficiency', 'cumulative_stability',
    'performance_gap', 'efficiency_score',
    'weighted_avg', 'improvement_rate'
]

In [35]:
model_data = features[final_features].copy()
model_data = model_data.rename(columns={'churn_target_first': 'churn_target'})
model_data = model_data.fillna(0)

print(f"Final dataset: {model_data.shape}")
print(f"Churn rate: {model_data['churn_target'].mean():.1%}")

Final dataset: (14583, 24)
Churn rate: 18.5%


In [37]:
# Feature analysis
for col in model_data.columns:
    if col != 'churn_target' and model_data[col].dtype in ['int64', 'float64']:
        churn_rate = model_data.groupby(col)['churn_target'].mean()
        if len(churn_rate) <= 10:  # Only show for categorical-like features
            print(f"{col}:")
            for val, rate in churn_rate.items():
                count = (model_data[col] == val).sum()
                print(f"  {val}: {rate:.1%} ({count:,} students)")

numeric_features = [col for col in model_data.columns if col not in ['churn_target', 'degree_id']]
correlations = model_data[numeric_features + ['churn_target']].corr()['churn_target'].abs().sort_values(ascending=False)
for feature, corr in correlations.head(8).items():
   if feature != 'churn_target':
       print(f"{feature}: {corr:.3f}")

is_remote:
  0: 15.0% (12,399 students)
  1: 38.2% (2,184 students)
is_bachelor:
  0: 9.5% (2,837 students)
  1: 20.7% (11,746 students)
is_female:
  0: 20.6% (6,990 students)
  1: 16.6% (7,593 students)
is_fall_start:
  0: 20.2% (2,808 students)
  1: 18.1% (11,775 students)
has_gap:
  0: 5.1% (10,823 students)
  1: 57.0% (3,760 students)
is_state_funded_first:
  0: 28.3% (5,143 students)
  1: 13.1% (9,440 students)
financial_status_changed:
  0: 18.3% (13,292 students)
  1: 20.9% (1,291 students)
is_from_budapest:
  0: 17.2% (9,201 students)
  1: 20.7% (5,382 students)
started_strong:
  0: 24.9% (8,767 students)
  1: 8.9% (5,816 students)
started_weak:
  0: 11.5% (10,025 students)
  1: 33.9% (4,558 students)
credit_efficiency: 0.679
activity_ratio: 0.665
has_gap: 0.585
credits_per_semester: 0.462
weighted_avg: 0.400
efficiency_score: 0.396
active_semesters_first: 0.372


In [38]:
model_data.to_csv('university_model.csv', index=False)